# GrayLog

In [33]:
from pygraylog.graylogapi import GraylogAPI
import json
import sys
import numpy as np
import pandas as pd
import datetime

In [27]:
log_level = 0

graylog_con={
        "user":"pavel",
        "password":"O777ver7k1",
        "url":"http://graylog-ui.roamability.com:9000/api",
        "offset":1,
        "limit":100
    }

api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()
        
fields=[
    'm3ua_OPC','m3ua_DPC',
    'sccp_calling_nai','sccp_calling_digits','sccp_calling_ri','sccp_calling_ssn',
    'sccp_called_nai','sccp_called_digits','sccp_called_ri','sccp_called_ssn',
    'sccp_class','sccp_message_type','protocol_name','protocol','protocol_version','opCode_localValue','TCAP_type',
    'tcap_otid','tcap_dtid',
    'e164_msisdn','e212_imsi']

In [18]:
#Load data from GrayLog

def LoadGraylogResult(exportsearch_results):
        jsonResult = json.loads(exportsearch_results)
        outputResult = list()
        i=0
        if "total_results" in jsonResult:
            outLog("Graylog search found {} messages".format(jsonResult["total_results"]),0)

        # Get the results and store them
            for result in jsonResult["messages"]:  
                if "message" in  result and isinstance(result["message"], dict):
                    outputResult.append(result["message"])
                    i+=1
                    if i%100==0: outLog("loaded {} records from Graylog".format(i),0)
            outLog("Graylog search load {} messages".format(len(outputResult)),0)
        else:
            outLog("Graylog search error: {}".format(exportsearch_results),0)
        return outputResult
    
#Load data into df

def LoadGraylogResultToDataFrame(data):
    columns=list(data[0].keys())
    values = list(data[0].values())
    arr_len = len(values)
    df=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
    for i in range(len(data)-1):
        columns=list(data[i+1].keys())
        values = list(data[i+1].values())
        arr_len = len(values)
        df1=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
        df=pd.concat([df, df1],axis=0,join='outer')
    df=df.reset_index(drop=True)
    if 'tcap_otid' in df.columns:
        df['tcap_otid']=df['tcap_otid'].fillna(0)
    if 'tcap_dtid' in df.columns:
        df['tcap_dtid']=df['tcap_dtid'].fillna(0)
    return df

#Compile transactions

def xdr_compile(otid,ind,df_xdr,df):
    
    fields=['tcap_dtid','tcap_otid','TCAP_type']
    if all(item in df.columns for item in fields):
        dtid=111
        df_xdr_temp=df_xdr
        for i in range(ind-1,-1,-1):
            if df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['tcap_otid']==0\
            and (df.iloc[i]['TCAP_type']=='End' or df.iloc[i]['TCAP_type']=='Abort'):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
                return df_xdr_temp
            elif df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['TCAP_type']=='Continue' and df.iloc[i]['tcap_otid']<>dtid:
                dtid=df.iloc[i]['tcap_otid']
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
            elif df.iloc[i]['TCAP_type']=='Continue'\
            and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
            elif (df.iloc[i]['TCAP_type']=='End' or df.iloc[i]['TCAP_type']=='Abort')\
            and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
                return df_xdr_temp
        return df_xdr_temp
    else:
        print 'Not all fields are in the DataFrame'

In [19]:
#--------------------------------------------------------------------------------
#Form date and time
#--------------------------------------------------------------------------------
#format like 2017-03-06T16:28

#Manually set the date and time
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')

#The last period of time
dateFrom = datetime.datetime.strptime((datetime.datetime.now()-datetime.timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M"),'%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M"), '%Y-%m-%dT%H:%M')

#--------------------------------------------------------------------------------
#Form query
#--------------------------------------------------------------------------------
#Fields examples
#protocol:ss7
#tcap_otid:"a2:2a:0c:40"
#tcap_dtid:"a2:2a:0c:40"
#opCode_localValue:2
#TCAP_type:Begin
#e212_imsi:425019629870706
#tcap_result:1
#tcap_dialogue_service_user:2

#TID
#query='protocol:ss7 AND (tcap_otid:"a2:2a:0c:40" OR tcap_dtid:"a2:2a:0c:40")'

#Opcode, TCAP_type and IMSI
#query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425019629870706'
#query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin'

#Wrong MTFSM MAP version
#query='protocol:ss7 AND tcap_result:1 AND tcap_dialogue_service_user:2 AND tcap_application_context_name:"0.4.0.0.1.0.25.2" AND (m3ua_OPC:1961 OR m3ua_OPC:1962)'

query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425019629817658'

#--------------------------------------------------------------------------------
#Run the query
#--------------------------------------------------------------------------------

gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo, offset=graylog_con["offset"], limit=graylog_con["limit"])
data=LoadGraylogResult(gres)

Graylog search found 3 messages
Graylog search load 2 messages


In [22]:
#Load data into df

try:
    df=LoadGraylogResultToDataFrame(data)
except:
    outLog('Nothing to combine.',0)  

xdr_fields=[]
for a in fields:
    if a in df.columns:
        #xdr_fields+=a
        xdr_fields.append(a)

df[xdr_fields].head()

,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,sccp_called_ssn,sccp_class,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,e164_msisdn,e212_imsi
0,1961,2505,4,447782000006,0,7,4,972549629817658,0,6,0,0x00000009,map,ss7,3,2,Begin,91:00:dc:fc,447782000006,425019629817658
1,1961,2506,4,447785014052,0,7,4,972549629817658,0,6,1,0x00000009,map,ss7,NaN,2,Begin,39:08:01:ec,447785014052,425019629817658


### Поиск в GrayLog по IMSI и выборка сообщений по TID

In [23]:
#The last period of time
dateFrom = datetime.datetime.strptime((datetime.datetime.now()-datetime.timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M"),'%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M"), '%Y-%m-%dT%H:%M')

#Opcode, TCAP_type and IMSI
#query='protocol:ss7 AND opCode_localValue:2* AND TCAP_type:Begin AND e212_imsi:425019629765531'
query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425019629817658'

#--------------------------------------------------------------------------------
#Run the query
#--------------------------------------------------------------------------------
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo, offset=graylog_con["offset"], limit=graylog_con["limit"])
data=LoadGraylogResult(gres)

Graylog search found 3 messages
Graylog search load 2 messages


In [24]:
tid_list=[]
str_temp=''

for i in range(len(data)):
    try:
        tid_list.append(data[i]['tcap_otid'])
    except:
        pass
    try:
        tid_list.append(data[i]['tcap_dtid'])
    except:
        pass
tid_list
tid_list=tuple(set(tid_list))

i=1
for a in tid_list:
    str_temp=str_temp + 'tcap_otid:' + '"' + a  + '"' + ' OR tcap_dtid:' + '"' + a  + '"'
    if i<len(tid_list):
        str_temp=str_temp + ' OR '
    i=i+1
query=str('protocol:ss7 AND (' + str_temp + ')')
query

'protocol:ss7 AND (tcap_otid:"39:08:01:ec" OR tcap_dtid:"39:08:01:ec" OR tcap_otid:"91:00:dc:fc" OR tcap_dtid:"91:00:dc:fc")'

In [25]:
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo, offset=graylog_con["offset"], limit=graylog_con["limit"])
data=LoadGraylogResult(gres)

try:
    df=LoadGraylogResultToDataFrame(data)
except:
    outLog('Nothing to combine.',0)  

Graylog search found 14 messages
Graylog search load 13 messages


In [28]:
#Сборка датафрейма XDR

fields_for_xdr=['tcap_dtid','tcap_otid','TCAP_type']

if all(item in df.columns for item in fields_for_xdr):
    columns=list(df.keys())
    df_xdr_res=pd.DataFrame(columns=columns)
    for i in range(len(df)-1,-1,-1):
        #if df.iloc[i]['TCAP_type']=='Begin' and (df.iloc[i]['e212_imsi']=='425019612512015'):
        if df.iloc[i]['TCAP_type']=='Begin':
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr_res=pd.concat([df_xdr_res,xdr_compile(df.iloc[i]['tcap_otid'],i,df_temp,df)],axis=0,join='outer')
else:
    print 'Not all fields are in the DataFrame'

df_xdr_res=df_xdr_res.reset_index(drop=True)

xdr_fields=[]
for a in fields:
    if a in df.columns:
        #xdr_fields+=a
        xdr_fields.append(a)

df_xdr_res[xdr_fields]

,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,sccp_called_ssn,...,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,1961,2506,4,447785014052,0,7,4,972549629817658,0,6,...,0x00000009,map,ss7,NaN,2,Begin,39:08:01:ec,0,447785014052,425019629817658
1,2506,1961,4,97254120624,0,6,4,447785014052,0,7,...,0x00000009,map,ss7,NaN,7,Continue,a2:1a:05:63,39:08:01:ec,972559395225,NaN
2,1961,2506,4,447785014052,0,7,4,97254120624,0,6,...,0x00000009,map,ss7,NaN,NaN,Continue,39:08:01:ec,a2:1a:05:63,NaN,NaN
3,2506,1961,4,97254120624,0,6,4,447785014052,0,7,...,0x00000009,map,ss7,NaN,7,Continue,a2:1a:05:63,39:08:01:ec,97254120624,NaN
4,1961,2506,4,447785014052,0,7,4,97254120624,0,6,...,0x00000009,map,ss7,NaN,7,Continue,39:08:01:ec,a2:1a:05:63,NaN,NaN
5,2506,1961,4,97254120624,0,6,4,447785014052,0,7,...,0x00000009,map,ss7,NaN,2,End,0,39:08:01:ec,97254120624,NaN
6,1961,2505,4,447782000006,0,7,4,972549629817658,0,6,...,0x00000009,map,ss7,3,2,Begin,91:00:dc:fc,0,447782000006,425019629817658
7,2505,1961,4,97254120624,0,6,4,447782000006,0,7,...,0x00000009,map,ss7,3,7,Continue,a1:1a:00:1d,91:00:dc:fc,972559395225,NaN
8,1962,2506,4,447782000006,0,7,4,97254120624,0,6,...,0x00000009,map,ss7,2,7,Continue,91:00:dc:fc,a1:1a:00:1d,NaN,NaN
9,2505,1961,4,97254120624,0,6,4,447782000006,0,7,...,0x00000009,map,ss7,NaN,7,Continue,a1:1a:00:1d,91:00:dc:fc,97254120624,NaN


# Splunk

In [34]:
from time import sleep
import sys
import splunklib.client as client

import numpy as np
import pandas as pd
import datetime

In [44]:
log_level = 0

def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

In [25]:
#Подключение к Splunk

#HOST = "localhost"
HOST = "splunk-search01.roamability.com"
PORT = 8089
USERNAME = "pavel"
PASSWORD = "O777ver7k1"
# Create a Service instance and log in 
service = client.connect(
    host=HOST,
    port=PORT,
    username=USERNAME,
    password=PASSWORD)
# Print installed apps to the console to verify login
for app in service.apps:
    print app.name

alert_logevent
alert_webhook
appsbrowser
framework
gettingstarted
introspection_generator_addon
launcher
learned
legacy
lookup_editor
sample_app
search
simple_xml_examples
splunk_app_db_connect
splunk_archiver
splunk_datasets_addon
splunk_httpinput
splunk_instrumentation
SplunkForwarder
SplunkLightForwarder


In [10]:
#Посмотреть сохраненные запросы

#List the saved searches that are available to the current user
savedsearches = service.saved_searches
for savedsearch in savedsearches:
    print "Search name: '" + savedsearch.name + "'"
    print "Query:\n" + savedsearch["search"] + "\n"

Search name: '0001 - MAP Incoming Errors'
Query:
index=sdr PROTOCOL=MAP (OPCODE=56 OR OPCODE=2*) SERVICE=dmi O_GT!=187638*
 | eval CGPA=substr(O_GT,1,6)
 | eval OK=mvfilter(match(RESULT,"ok"))
 | eval Timer=mvfilter(match(RESULT,"timeout"))
 | eval RNA=mvfilter(like(RESULT,"%Roaming%llowed%"))
 | eval Unknown_sr=mvfilter(like(RESULT,"%nknown%ubscriber%"))
| stats count as Total ,count(OK) as OK, count(Timer) as Timer, count(Unknown_sr) as Unknown_sr, count(RNA) as RNA
by CGPA | eval ErrorsPercentage=(if(Total=0,0,round((Total-OK-RNA-Unknown_sr)/Total*100,0))) | where ErrorsPercentage > 20 | where Total >100 | sort ErrorsPercentage desc

Search name: '0002 - MAP Outgoing Errors'
Query:
index=sdr PROTOCOL=MAP SERVICE=stp (OPCODE=3 OR OPCODE=4 OR (OPCODE=44 RESULT!="T_Abort")) D_GT!=972559900040
| eval CDPA=substr(D_GT,1,6)
| eval OK=mvfilter(match(RESULT,"ok"))
| eval TError=mvfilter(match(RESULT,"T_Error"))
| eval TAbort=mvfilter(match(RESULT,"T_Abort"))
| eval Timer=mvfilter(match(RESU

In [19]:
#Создать запрос

# Delete the test search
service.saved_searches.delete("Test Search")
# Create a saved search--search everything, return 1st 10 events
# Note: Do not include the 'search' keyword for a saved search
myquery = "index=sdr | head 1"
mysearchname = "Test Search"
mysavedsearch = service.saved_searches.create(mysearchname, myquery)
print "Created Search name: '" + mysavedsearch.name + "'"
print "Query:\n" + mysavedsearch["search"]

Created Search name: 'Test Search'
Query:
index=sdr | head 1


In [20]:
# Retrieve the new search
mysavedsearch = service.saved_searches["Test Search"]
# Run the saved search
job = mysavedsearch.dispatch()
# Create a small delay to allow time for the update between server and client
sleep(2)
# Wait for the job to finish--poll for completion and display stats
while True:
    job.refresh()
    stats = {"isDone": job["isDone"],
             "doneProgress": float(job["doneProgress"])*100,
              "scanCount": int(job["scanCount"]),
              "eventCount": int(job["eventCount"]),
              "resultCount": int(job["resultCount"])}
    status = ("\r%(doneProgress)03.1f%%   %(scanCount)d scanned   "
              "%(eventCount)d matched   %(resultCount)d results") % stats
    sys.stdout.write(status)
    sys.stdout.flush()
    if stats["isDone"] == "1":
        break
    sleep(2)
# Display the search results now that the job is done
jobresults = job.results()
while True:
    content = jobresults.read(1024)
    if len(content) == 0: break
    sys.stdout.write(content)
    sys.stdout.flush()
sys.stdout.write("\n")

100.0%   104 scanned   1 matched   1 results<?xml version='1.0' encoding='UTF-8'?>
<results preview='0'>
<meta>
<fieldOrder>
<field>_bkt</field>
<field>_cd</field>
<field>_indextime</field>
<field>_raw</field>
<field>_serial</field>
<field>_si</field>
<field>_sourcetype</field>
<field>_time</field>
<field>host</field>
<field>index</field>
<field>linecount</field>
<field>source</field>
<field>sourcetype</field>
<field>splunk_server</field>
</fieldOrder>
</meta>
	<result offset='0'>
		<field k='_bkt'>
			<value><text>sdr~59~D3D776C3-EC79-446F-B168-96EA8C262D36</text></value>
		</field>
		<field k='_cd'>
			<value><text>59:60314010</text></value>
		</field>
		<field k='_indextime'>
			<value><text>1521178456</text></value>
		</field>
		<field k='_raw'><v xml:space='preserve' trunc='0'>16/03/2018 06:34:16; SERVICE=&quot;ss7fep&quot;; ID=&quot;68&quot;; EVENT=&quot;Add new SSN FEP definition: ssn=6, inst=1&quot;</v></field>
		<field k='_serial'>
			<value><text>0</text></value>
		</field>
	

In [68]:
import splunklib
import splunklib.client as client
import splunklib.results as results

def FormatSplunkEngineDate(dt): return dt.strftime("%Y-%m-%dT%H:%M:%S")
def FormatGraylogEngineDate(dt): return dt.strftime("%Y-%m-%dT%H:%M:%S.000Z")

def LoadSplunkResult(exportsearch_results):
        outputResult = list()
        i=0
    # Get the results and store them
        reader = results.ResultsReader(exportsearch_results)
        for result in reader:   
            if isinstance(result, dict):
                outputResult.append(result)
                i+=1
                if i%100==0: outLog("loaded {} records from splunk".format(i),0)
            elif isinstance(result, results.Message):
                # Diagnostic messages may be returned in the results
                outLog("Message: %s" % result)
        outLog("Splunk search found {} messages".format(len(outputResult)),0)
        return outputResult

def getSplunkFileResult(service, dateFrom, dateTo,fileMask=None,protocol=None, bEchoTime = False):
    # Run an export search and display the results using the results reader.
    # Set the parameters for the search:
    sfrom = dateFrom if  bEchoTime else FormatSplunkEngineDate(dateFrom) 
    sto = dateTo if  bEchoTime else FormatSplunkEngineDate(dateTo)
    kwargs_export = {"earliest_time": sfrom,
                     "latest_time":sto ,
                     "search_mode": "normal",
                     #,"output_mode":"JSON"
                     #"rf" : fields,
                     "supports_multivalues":True
                     }

    #searchquery_export = 'search index=pcap protocol={} {} | extract mv_add=true , pairdelim="\\t;", kvdelim="=" | stats  min(frame_time_epoch) as minTime max(frame_time_epoch) as maxTime  by file '.format(protocol,fileMask)
    #searchquery_export = 'search index=sdr | head 1'
    searchquery_export = 'search index=sdr PROTOCOL=MAP IN_IMSI=425019611031032 OPCODE=56 | head 1'
    outLog("Run Splunk file search by query {}".format(searchquery_export),0)

    exportsearch_results = service.jobs.export(searchquery_export, **kwargs_export)

    return LoadSplunkResult(exportsearch_results)

def getSplunkSearchResult(service, dateFrom, dateTo, searchQuery,fileMask=None,protocol=None,fields=None, bEchoTime = False):
    # Run an export search and display the results using the results reader.
    # Set the parameters for the search:
    sfrom = dateFrom if  bEchoTime else FormatSplunkEngineDate(dateFrom) 
    sto = dateTo if  bEchoTime else FormatSplunkEngineDate(dateTo)
    kwargs_export = {"earliest_time": sfrom,
                     "latest_time":sto ,
                     "search_mode": "normal",
                     #,"output_mode":"JSON"
                     "rf" : fields,
                     "supports_multivalues":True
                     }
    fieldsStr = ','.join(fields)
    
    #searchquery_export = 'search index=pcap protocol={} {} {} | extract mv_add=true , pairdelim="\\t;", kvdelim="=" |  fields {}'.format(protocol,fileMask,searchQuery,fieldsStr)
    searchquery_export = 'search index=sdr | head 1'
    outLog("Run Splunk search by query {}".format(searchquery_export))
    exportsearch_results = service.jobs.export(searchquery_export, **kwargs_export)

    return LoadSplunkResult(exportsearch_results)

def runSplunkSearch(filterObj,rangeObj,protocol,fileList,iterations,outputFolder):

    outLog("Run search by protocol " + protocol,0)
    outputSplunk = outputFolder +"/splunk."+ protocol + ".pcap"
    timeRange = GetTimeRange(protocol)

    service = client.connect(host='splunk.roamability.com', port=8089,username="admin", password='3dFrcWbJQ7Ap7AvemXifxgEP97VQHEHdKVU')
    splunkMask = GetFilterMask(filterObj,True,False,False)
    customMask = GetFilterMask(filterObj,False,False,True)
    if splunkMask=="" and customMask!="":
        outLog("Skip search splunk by protocol " + protocol + " due only custom filter set",0)
        return outputSplunk;
    fileMask = ""
    #add files to mask
    firstFile=True
    fileMask ="("
    for fname in fileList:
        if not fileList[fname]["enabled"]: continue
        if fileList[fname]["protocol"]!=protocol: continue
        if not firstFile: fileMask+=" OR "
        fileMask+="file={}".format(fileList[fname]["splunkmask"])
        firstFile = False
    fileMask+=")"
    if(firstFile): return outputSplunk


    #format like 2017-03-06T16:28
    dateFrom = datetime.datetime.strptime(rangeObj["From"], '%Y-%m-%dT%H:%M')
    dateTo = datetime.datetime.strptime(rangeObj["To"], '%Y-%m-%dT%H:%M')
    
    fields = ["file","frame_time_epoch"] + getProtocolFilterField(protocol)
    if splunkMask=="":
        splunkResult = getSplunkFileResult(service,dateFrom,dateTo,fileMask,protocol)
        if len(splunkResult) == 0:
            outLog('Empty file list after filtering. Exiting...'.format(protocol),0)
            return outputSplunk
        fileList = getFileListRange(splunkResult)
        outLog(fileList)
        mergeOutputPlain(outputSplunk,fileList,outputFolder,protocol)
    else:
        splunkResult = getSplunkSearchResult(service,dateFrom,dateTo,splunkMask,fileMask,protocol,fields)
        if len(splunkResult) == 0:
            outLog('Empty transactions list after filtering. Exiting...'.format(protocol),0)
            return outputSplunk
        #we get results, then we have to go by TIDS
        i = 0
    
        while i < iterations:
            tids = getTidsDict(splunkResult,protocol)
            outLog("{} messages ids found before {} iterations".format(len(tids),i + 1),0)

            #delta = datetime.timedelta(seconds=timeRange * (i + 1))
            delta = timeRange * (i + 1)
            if(len(tids) > 0):
                tidMask, dtFrom, dtTo = createTidMask(tids,protocol,timeRange,False)
                if tidMask=='': 
                    outLog('Empty {} transactions list after filtering. Exiting...'.format(protocol),0)
                    return outputSplunk

                splunkResult = getSplunkSearchResult(service,dtFrom - delta, dtTo + delta,tidMask,fileMask,protocol,fields,True)
                i+=1
            
            else:
                outLog('Empty {} transactions list after filtering. Exiting...'.format(protocol),0)
                return outputSplunk

        fileList = getFileList(splunkResult,protocol)

        outLog(fileList)
        mergeOutput(outputSplunk,fileList,outputFolder)
    return outputSplunk


In [69]:
dateFrom = datetime.datetime.strptime('2018-03-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-03-12T05:10', '%Y-%m-%dT%H:%M')

#sfrom = dateFrom if  bEchoTime else FormatSplunkEngineDate(dateFrom) 
#sto = dateTo if  bEchoTime else FormatSplunkEngineDate(dateTo)
sfrom = FormatSplunkEngineDate(dateFrom) 
sto = FormatSplunkEngineDate(dateTo)

res=getSplunkFileResult(service,dateFrom,dateTo)
res

Run Splunk file search by query search index=sdr PROTOCOL=MAP IN_IMSI=425019611031032 OPCODE=56 | head 1
Splunk search found 1 messages


[OrderedDict([('IN_IMSI', '425019611031032'),
              ('OPCODE', '56'),
              ('PROTOCOL', 'MAP'),
              ('_bkt', 'sdr~57~AB9CA7E0-957E-4F84-9840-0E7D97578CB1'),
              ('_cd', '57:22908727'),
              ('_indextime', '1520827729'),
              ('_kv', '1'),
              ('_raw',
               '12/03/2018 05:07:51.043;\t SERVICE="stp";\tSEP_RI="63";\tGW_REQUEST="1";\tORIGINATION_SEP="393";\tPROTOCOL="MAP";\tIN_IMSI="425019611031032";\tO_GT="8613441822";\tD_GT="972549611031032";\tO_SSN="7";\tD_SSN="6";\tOPCODE="56";\tACN="04000001000e03";\tPREFIX_FLAG="1";\tDESTINATION="multibyte";\tRESULT="ok";\tDURATION="0.290"'),
              ('_serial', '0'),
              ('_si', ['splunk-idx-01', 'sdr']),
              ('_sourcetype', 'sdr-logs'),
              ('_time', '2018-03-12 05:07:51.000 CET'),
              ('host', 'stp1'),
              ('index', 'sdr'),
              ('linecount', '1'),
              ('source', '/tmp/sdr/sdr_1520827729_63.txt'),
  

In [79]:
res[0]['IN_IMSI']

'425019611031032'